In [1]:
import json
import os
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import json
import os.path as osp

In [3]:
data_path = '..//gen_exp_5_model/results/all'
data_path_steam = osp.join(data_path, 'steam.csv')
data_path_ml = osp.join(data_path, 'ml-100k.csv')
data_path_mind = osp.join(data_path, 'mind_small_dev.csv')

data_steam = pd.read_csv(data_path_steam)
data_ml = pd.read_csv(data_path_ml)
data_mind = pd.read_csv(data_path_mind)
data_steam.head(3)

,instruction,LLaMA2,ChatGLM2,GPT3.5,GPT4,LLaMA2-SFT
0,The history games played by the customer are:\...,"Based on your historical play record, Rust is ...","I'm sorry, but I am unable to determine the cu...","Dear customer, you should play Rust because it...","Based on your history, you enjoy action and ad...","Based on your history, you enjoy action, adven..."
1,The history games played by the customer are:\...,"Based on your historical play record, Rust is ...","I'm sorry, but I am unable to determine the cu...","Based on your historical play record, you shou...","You've shown interest in Action, Adventure, In...","Based on your history, you enjoy action and ad..."
2,The history games played by the customer are:\...,"Based on your historical play record, Team For...",You play Team Fortress 2 because it is an acti...,"Dear customer, you should play Team Fortress 2...","Given your history, you'd enjoy Team Fortress ...","Based on your history, you enjoy action games ..."


In [4]:
import json
import os.path as osp
from typing import Union

def get_instruction(instruction, exp1, exp2):
    prompt = "-"*20 + "Instruction" + "-"*20 + "\n"
    prompt += instruction + "\n"
    prompt += "-"*20 + "Explanation 1" + "-"*20 + "\n"
    prompt += exp1 + "\n"
    prompt += "-"*20 + "Explanation 2" + "-"*20 + "\n"
    prompt += exp2 + "\n"
    prompt += "-"*53 + "\n"

    return prompt

class Prompter_GPT(object):
    __slots__ = ("template", "_verbose")

    def __init__(self, verbose: bool = False):
        super().__init__()

    def generate_prompt(
        self,
        instruction: str,
        input: Union[None, str] = None,
    ) -> str:
        if input:
            prompt = instruction + input
        else:
            prompt = instruction

        prompt_template = \
f"You are a discriminator that judges whether the explainability of the recommendation system is good or bad. You should judge which of the 2 interpretability opinions generated based on the following Instruction is better. Return 1 if you think the first one is better, and 2 if you think the second one is better. Only the number 1 or 2 should be returned. Do not return any other characters.\n\n{prompt}Based on the above instructions, decide which explanation better explains why the recommendation system recommends this item to the customer.Please return 1 or 2 to show your choice. Only return 1 or 2. Do not return any other information."

        return prompt_template

    def get_response(self, output: str) -> str:
        return output

prompter = Prompter_GPT()

In [5]:
from tqdm import tqdm
def gen_compare_test_sft_data(df, model1, model2, f):
    for i in tqdm(range(df.shape[0])):
        data = df.iloc[i]
        instruction = data['instruction']
        exp1 = data[model1]
        exp2 = data[model2]
        instruction = get_instruction(instruction, exp1, exp2)

        json_data = {}
        json_data['instruction'] = instruction
        json_data['input'] = ''
        json_data['output'] = ''

        json.dump(json_data, f, ensure_ascii=False)
        f.write('\n')
    return json_data

In [6]:
import os
import pandas as pd

if not os.path.exists('data/'):
    os.makedirs('data/steam/')
    os.makedirs('data/ml/')
    os.makedirs('data/mind/')
    os.makedirs('data/compare_test/steam/')
    os.makedirs('data/compare_test/ml/')
    os.makedirs('data/compare_test/mind/')


def gen_dataset_model_compare(dataset_name, model1, model2):
    if dataset_name == 'mind':
        df = data_mind
    elif dataset_name == 'ml':
        df = data_ml
    elif dataset_name == 'steam':
        df = data_steam
    else:
        raise ValueError('dataset_name should be one of mind, ml, steam')

    with open(f'data/compare_test/{dataset_name}/{model1}-{model2}.json', 'w', encoding='utf-8') as f:
        f.close()
    with open(f'data/compare_test/{dataset_name}/{model1}-{model2}.json', 'a+', encoding='utf-8') as f:
        gen_compare_test_sft_data(df, model1, model2, f)
        f.close()


# Generate MIND data for all models

In [7]:
for model1, model2 in[('LLaMA2', 'ChatGLM2'),
                      ('LLaMA2', 'GPT3.5'),
                      ('LLaMA2', 'GPT4'),
                      ('LLaMA2', 'LLaMA2-SFT'),
                      ('ChatGLM2', 'GPT3.5'),
                      ('ChatGLM2', 'GPT4'),
                      ('ChatGLM2', 'LLaMA2-SFT'),
                      ('GPT3.5', 'GPT4'),
                      ('GPT3.5', 'LLaMA2-SFT'),
                      ('GPT4', 'LLaMA2-SFT')]:
    gen_dataset_model_compare('mind', model1, model2)

100%|██████████| 1000/1000 [00:00<00:00, 15992.19it/s]


# Generate ML data for all models

In [8]:
for model1, model2 in [('LLaMA2', 'ChatGLM2'),
                       ('LLaMA2', 'GPT3.5'),
                       ('LLaMA2', 'GPT4'),
                       ('LLaMA2', 'LLaMA2-SFT'),
                       ('ChatGLM2', 'GPT3.5'),
                       ('ChatGLM2', 'GPT4'),
                       ('ChatGLM2', 'LLaMA2-SFT'),
                       ('GPT3.5', 'GPT4'),
                       ('GPT3.5', 'LLaMA2-SFT'),
                       ('GPT4', 'LLaMA2-SFT')]:
    gen_dataset_model_compare('ml', model1, model2)

100%|██████████| 943/943 [00:00<00:00, 11075.50it/s]


# Generate Steam data for all models

In [9]:
for model1, model2 in [('LLaMA2', 'ChatGLM2'),
                       ('LLaMA2', 'GPT3.5'),
                       ('LLaMA2', 'GPT4'),
                       ('LLaMA2', 'LLaMA2-SFT'),
                       ('ChatGLM2', 'GPT3.5'),
                       ('ChatGLM2', 'GPT4'),
                       ('ChatGLM2', 'LLaMA2-SFT'),
                       ('GPT3.5', 'GPT4'),
                       ('GPT3.5', 'LLaMA2-SFT'),
                       ('GPT4', 'LLaMA2-SFT')]:
    gen_dataset_model_compare('steam', model1, model2)

100%|██████████| 1000/1000 [00:00<00:00, 15999.63it/s]


# 数据处理

In [11]:
dataset = ['ml', 'mind', 'steam']
train_ratio = 0.8

train_data = []
df = pd.DataFrame(columns=['index', 'dataset', 'model1', 'model2'])
index = 0

for dataset_name in dataset:
    print(dataset_name)
    for model1, model2 in [('LLaMA2', 'ChatGLM2'),
                           ('LLaMA2', 'GPT3.5'),
                           ('LLaMA2', 'GPT4'),
                           ('LLaMA2', 'LLaMA2-SFT'),
                           ('ChatGLM2', 'GPT3.5'),
                           ('ChatGLM2', 'GPT4'),
                           ('ChatGLM2', 'LLaMA2-SFT'),
                           ('GPT3.5', 'GPT4'),
                           ('GPT3.5', 'LLaMA2-SFT'),
                           ('GPT4', 'LLaMA2-SFT')]:
        print(model1, model2)
        with open(f'data/compare_test/{dataset_name}/{model1}-{model2}.json', 'r', encoding='utf-8') as f:
            data = f.readlines()
            for line in data[: int(train_ratio*len(data))]:
                train_data.append(line)
                index += 1
                df.loc[index] = [index, dataset_name, model1, model2]
            f.close()

with open(f'data/compare_test/all/train.json', 'w', encoding='utf-8') as f:
    f.writelines(train_data)
    f.close()

ml
LLaMA2 ChatGLM2
LLaMA2 GPT3.5
LLaMA2 GPT4
LLaMA2 LLaMA2-SFT
ChatGLM2 GPT3.5
ChatGLM2 GPT4
ChatGLM2 LLaMA2-SFT
GPT3.5 GPT4
GPT3.5 LLaMA2-SFT
GPT4 LLaMA2-SFT
mind
LLaMA2 ChatGLM2
LLaMA2 GPT3.5
LLaMA2 GPT4
LLaMA2 LLaMA2-SFT
ChatGLM2 GPT3.5
ChatGLM2 GPT4
ChatGLM2 LLaMA2-SFT
GPT3.5 GPT4
GPT3.5 LLaMA2-SFT
GPT4 LLaMA2-SFT
steam
LLaMA2 ChatGLM2
LLaMA2 GPT3.5
LLaMA2 GPT4
LLaMA2 LLaMA2-SFT
ChatGLM2 GPT3.5
ChatGLM2 GPT4
ChatGLM2 LLaMA2-SFT
GPT3.5 GPT4
GPT3.5 LLaMA2-SFT
GPT4 LLaMA2-SFT


In [12]:
dataset = ['ml', 'mind', 'steam']
train_ratio = 0.8

test_data = []
df = pd.DataFrame(columns=['index', 'dataset', 'model1', 'model2'])
index = 0

for dataset_name in dataset:
    print(dataset_name)
    for model1, model2 in [('LLaMA2', 'ChatGLM2'),
                           ('LLaMA2', 'GPT3.5'),
                           ('LLaMA2', 'GPT4'),
                           ('LLaMA2', 'LLaMA2-SFT'),
                           ('ChatGLM2', 'GPT3.5'),
                           ('ChatGLM2', 'GPT4'),
                           ('ChatGLM2', 'LLaMA2-SFT'),
                           ('GPT3.5', 'GPT4'),
                           ('GPT3.5', 'LLaMA2-SFT'),
                           ('GPT4', 'LLaMA2-SFT')]:
        print(model1, model2)
        with open(f'data/compare_test/{dataset_name}/{model1}-{model2}.json', 'r', encoding='utf-8') as f:
            data = f.readlines()
            for line in data[int(train_ratio*len(data)):]:
                test_data.append(line)
                index += 1
                df.loc[index] = [index, dataset_name, model1, model2]
            f.close()


ml
LLaMA2 ChatGLM2
LLaMA2 GPT3.5
LLaMA2 GPT4
LLaMA2 LLaMA2-SFT
ChatGLM2 GPT3.5
ChatGLM2 GPT4
ChatGLM2 LLaMA2-SFT
GPT3.5 GPT4
GPT3.5 LLaMA2-SFT
GPT4 LLaMA2-SFT
mind
LLaMA2 ChatGLM2
LLaMA2 GPT3.5
LLaMA2 GPT4
LLaMA2 LLaMA2-SFT
ChatGLM2 GPT3.5
ChatGLM2 GPT4
ChatGLM2 LLaMA2-SFT
GPT3.5 GPT4
GPT3.5 LLaMA2-SFT
GPT4 LLaMA2-SFT
steam
LLaMA2 ChatGLM2
LLaMA2 GPT3.5
LLaMA2 GPT4
LLaMA2 LLaMA2-SFT
ChatGLM2 GPT3.5
ChatGLM2 GPT4
ChatGLM2 LLaMA2-SFT
GPT3.5 GPT4
GPT3.5 LLaMA2-SFT
GPT4 LLaMA2-SFT


In [13]:
df.to_csv('data/compare_test/all/test_index.csv', index=False)
with open(f'data/compare_test/all/test.json', 'w', encoding='utf-8') as f:
    f.writelines(test_data)
    f.close()